In [3]:
import openai

In [5]:
!pip install praw

  Using cached praw-7.7.0-py3-none-any.whl (189 kB)
  Using cached prawcore-2.3.0-py3-none-any.whl (16 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: c:\Users\sohai\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [6]:
import praw
import os

In [7]:
os.environ['REDDIT_CLI_ID'] = 'kmuLnqwwYhxGWc73vK7Qyg'
os.environ['REDDIT_CLI_SEC'] = 'MPVfxdFw0AzhYBWlUHQ0DFieaEy2RQ'

In [8]:
Cli_ID = os.getenv('REDDIT_CLI_ID')

Cli_Sec = os.getenv('REDDIT_CLI_SEC')

In [9]:
reddit = praw.Reddit(client_id= Cli_ID,
                     client_secret = Cli_Sec,
                     user_agent = 'Sentiment Analysis Test'
                     )

In [10]:
for submissions in reddit.subreddit('finance').hot(limit = 5):
    print(submissions.title)

Moronic Monday - July 04, 2023 - Your Weekly Questions Thread
We are Wall Street Journal reporters covering the collapse of SVB. Ask us anything.
Out-of-sync U.S. stocks hide market risks
Recession canceled? U.S. stock market ‘pretty frothy’ after S&P 500’s strongest first half since 2019
The End of LIBOR Is (Finally) Here


In [11]:
subreddit_finance = reddit.subreddit('finance')

In [12]:
subreddit_finance.accounts_active

443

In [13]:
subreddit_stocks = reddit.subreddit('stocks')
subreddit_stocks.accounts_active

4288

In [14]:
for post in subreddit_stocks.hot(limit = 5):
    print(post.title)
    submission = reddit.submission(post.id)
    counter = 0
    for comment in submission.comments:
        if comment.body == '[deleted]':
            pass
        counter += 1
        if counter == 2:
            break

FYI Apollo/RIF users of r/Stocks, click here - Reddit valuation/IPO discussion inside
r/Stocks Daily Discussion Wednesday - Jul 05, 2023
A U.S.-China tech war will hurt the rally in tech stocks
Congressional Insider Trading in 2022: Using Inside Knowledge to Make Profits
Is it a good time to buy bank stocks?


In [15]:
def get_titles_comments(subreddit = 'stocks', limit=6, num_comments=3,skip_pinned = 2):
    subreddit = reddit.subreddit(subreddit)
    title_comments = {}

    for counter,post in enumerate(subreddit.hot(limit = limit)):
        if counter < skip_pinned:
            continue
        counter += (1-skip_pinned)
        title_comments[counter] = ""
        subm = reddit.submission(post.id)
        title = post.title
        title_comments[counter] += 'Title: '+title+"\n\n"
        title_comments[counter] += 'Comments: \n\n'

        comn_counter = 0
        for comment in subm.comments:
            if not comment.body == '[deleted]':
                title_comments[counter] += comment.body+"\n"
                comn_counter += 1
            if comn_counter == num_comments:
                break
    return title_comments

In [16]:
titles_comments = get_titles_comments()

In [18]:
print(titles_comments)

{1: 'Title: Congressional Insider Trading in 2022: Using Inside Knowledge to Make Profits\n\nComments: \n\nIt makes me vomit as an American that this garbage is allowed.\nLet’s talk about how Supreme Court decisions are for sale…\nAt what point does a line get crossed when it comes to corruption?\n', 2: 'Title: A U.S.-China tech war will hurt the rally in tech stocks\n\nComments: \n\nHi, you\'re on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.\n\n**To everyone commenting:**  Please focus on how this affects the stock market or specific stocks or it will be removed as being off-topic/political.\n\nIf you\'re interested in just politics, see our wiki on ["relevant subreddits"](https://www.reddit.com/r/stocks/wiki/index) and post to those Reddit communities instead without linking back here, thanks!\n\n*I am a bot, and this action was performed aut

In [19]:
def create_prompt(title_comments):
     task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
     return task + title_comments


In [20]:
prompt = create_prompt(titles_comments[1])

In [21]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: Congressional Insider Trading in 2022: Using Inside Knowledge to Make Profits

Comments: 

It makes me vomit as an American that this garbage is allowed.
Let’s talk about how Supreme Court decisions are for sale…
At what point does a line get crossed when it comes to corruption?



In [23]:
for key, t_n_c in titles_comments.items():

    prompt = create_prompt(t_n_c)

    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(t_n_c)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)

Title: Congressional Insider Trading in 2022: Using Inside Knowledge to Make Profits

Comments: 

It makes me vomit as an American that this garbage is allowed.
Let’s talk about how Supreme Court decisions are for sale…
At what point does a line get crossed when it comes to corruption?

Sentiment: 
No company mentioned
------------------------------
Title: A U.S.-China tech war will hurt the rally in tech stocks

Comments: 

Hi, you're on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.

**To everyone commenting:**  Please focus on how this affects the stock market or specific stocks or it will be removed as being off-topic/political.

If you're interested in just politics, see our wiki on ["relevant subreddits"](https://www.reddit.com/r/stocks/wiki/index) and post to those Reddit communities instead without linking back here, thanks!

*I am a bot,